In [1]:
import sys
sys.path.append("../library/")
from dict_net import *
from deep_dream import *
from helper_functions import *
from discrim_net import *
from glue_layers import *
import matplotlib.pyplot as plt

In [2]:
# set dreamer network
net = DictNet(1000)
net.load_state_dict(torch.load("../models/net_1000_0.001_200_0.0.pth",map_location=torch.device('cpu')))

<All keys matched successfully>

In [3]:
#set glue layer
glue_net = BasicGlueLayer()

In [4]:
#Initialize dreamer
dreamer = DeepDreamGAN(net,(1,32,128),(0.47,),(0.14,),use_gaussian_filter=True,glue_layer=glue_net)

Device used to run this program:  cpu
Loading the network...
Network Loaded
Gaussian filter will be used
Discriminator initialized with DiscrimNet with 2 final outputs


In [5]:
#set dataset
data_root = "../dataset/"
num_labels = 1000
transform = dg.mjsynth.mjsynth_gray_scale
ds, labels_and_indices_dict, labels_dict, labels_inv_dict = create_dicts(data_root,transform)
ds = extract_dataset(ds,labels_and_indices_dict,labels_dict,num_labels,prev_num_labels=0)

In [8]:
#train dreamer
batch_size = 4
lr = 0.001
trainloader = torch.utils.data.DataLoader(ds, batch_size=batch_size,shuffle=True) 

criterion_discrim = nn.CrossEntropyLoss()
optimizer_discrim = torch.optim.Adam(dreamer.discrim_net.parameters(), lr=lr)

criterion_glue_layer = nn.CrossEntropyLoss()
optimizer_glue_layer = torch.optim.Adam(dreamer.glue_layer.parameters(), lr=lr)

In [9]:
for i,data in enumerate(trainloader,0):
    images,targets = dreamer.create_data_and_targets(data,batch_size) 
    
    optimizer_discrim.zero_grad()
    outputs = dreamer.discrim_net(images)

    loss_discrim = criterion_discrim(outputs, targets)
    loss_discrim.backward()
    optimizer_discrim.step()

    print("Discriminator loss : ",loss_discrim)
    break

Dreaming a batch..


KeyboardInterrupt: 